In [6]:
# NOTE: you CAN change this cell
# If you want to use your own database, download it here
# !gdown ...
!rm -rf list_province.txt
!rm -rf list_district.txt
!rm -rf list_ward.txt

!gdown --fuzzy https://drive.google.com/file/d/1FC5Kb1iL_ElLboQ_yuJt-RyujZWVBe5N/view?usp=sharing -O list_province.txt
!gdown --fuzzy https://drive.google.com/file/d/1mLEuPqn_01ffMGnDy2fntHhpWSNyYHSW/view?usp=sharing -O list_district.txt
!gdown --fuzzy https://drive.google.com/file/d/1LmCBNSqf2fY4lcphAFgQlMR_GkCwSLkk/view?usp=sharing -O list_ward.txt

Downloading...
From: https://drive.google.com/uc?id=1FC5Kb1iL_ElLboQ_yuJt-RyujZWVBe5N
To: /content/list_province.txt
100% 5.07k/5.07k [00:00<00:00, 18.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mLEuPqn_01ffMGnDy2fntHhpWSNyYHSW
To: /content/list_district.txt
100% 4.72k/4.72k [00:00<00:00, 16.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LmCBNSqf2fY4lcphAFgQlMR_GkCwSLkk
To: /content/list_ward.txt
100% 4.79k/4.79k [00:00<00:00, 16.5MB/s]


In [7]:
  # NOTE: you CAN change this cell
# Add more to your needs
# you must place ALL pip install here
!pip install editdistance

In [8]:
# NOTE: you CAN change this cell
# import your library here
import time
import pandas as pd
import re
import math
import unicodedata
import json
from collections import Counter
from typing import Dict, Tuple, Optional, List
from queue import Queue
from datetime import datetime
from collections import Counter
import editdistance
from gettext import textdomain
from numpy import character


# Utils

In [9]:
import re
from gettext import textdomain

import unicodedata
from numpy import character

SAFE_CASES = [
                #"TP.", "T.P", "F.", "Thành phố", "ThànhPhố", "TP ", " TP",
               "Tỉnh", " T ", #"Tỉn",  ",T ", "T.",
               #"Quận", "Q.", " Q ", ",Q ", -> quận 5 sau khi remove sẽ thành 5 và không tìm ra
               "Huyện", "hyện", #"H.", " H ", ",H ",
               # "Phường", "F",  "P.", " P ", ",P ",
               "Thị xã", "ThịXã", "Xã", # "X.", " X ", "X ", ",X ",
                "Thị trấn", "ThịTrấn", #"T.T",
                "khu phố", "KP", "KhuPhố", "Khu pho", "KhuPho", # -> KP5 bị nhầm thành P5
                "Ấp"
               ]


ALL_PREFIXES = ["TP.", "T.P", "F.", "Thành phố", "ThànhPhố", "TP ", " TP",
               "Tỉnh", "Tỉn","T.", " T ",  #",T ",
               "Quận", "Q.", " Q ", ",Q ",
               "Huyện", "hyện", "H.", " H ",  #",H ",
                #"KP", # KP. contains P.
               "khu phố",  "KhuPhố", "Khu pho", "KhuPho",
               "Phường", "P.", "F", " P ",  #",P ",
               "X.", "Thị xã", "ThịXã", "Xã",  #" X ", "X ", ",X ",
                "Thị trấn", "ThịTrấn", "T.T",
               "-"
                ]

vietnamese_dict = {
    "a": ["à", "á", "ạ", "ả", "ã", "â", "ầ", "ấ", "ậ", "ẩ", "ẫ", "ă", "ằ", "ắ", "ặ", "ẳ", "ẵ"],
    "d": ["đ"],
    "e": ["è", "é", "ẹ", "ẻ", "ẽ", "ê", "ề", "ế", "ệ", "ể", "ễ"],
    "i": ["ì", "í", "ị", "ỉ", "ĩ"],
    "o": ["ò", "ó", "ọ", "ỏ", "õ", "ô", "ồ", "ố", "ộ", "ổ", "ỗ", "ơ", "ờ", "ớ", "ợ", "ở", "ỡ"],
    "u": ["ù", "ú", "ụ", "ủ", "ũ", "ư", "ừ", "ứ", "ự", "ử", "ữ"],
    "y": ["ỳ", "ý", "ỵ", "ỷ", "ỹ"],

}

reversed_vietnamese_dict = {}

wrong_accents = {
    "oà": "òa", "oá": "óa", "oạ": "ọa", "oã": "õa", "oả": "ỏa",
    "qui": "quy",
}

def common_normalize(text: str) -> str:
    text = text.lower()
    # text = text.replace(",", "")  # replace for T,â,n,B,ì,n,h Dĩ An Bình Dương
    # text = text.replace(".", "")
    for case in SAFE_CASES:
        text = re.sub(re.escape(case), ',', text, flags=re.IGNORECASE)
    text = re.sub(r"\s{2,}", ",", text)  # Remove spaces
    text = text.replace(" ", "")
    return text

def normalize_text_and_remove_accent(text: str) -> str:
    """Normalize text by removing accents, spaces, and special cases."""
    text = common_normalize(text)
    text = unicodedata.normalize("NFKD", text)
    text = text.replace("đ", "d")
    text = "".join(c for c in text if not unicodedata.combining(c))  # Remove accents
    # text = re.sub(r"\s+", "", text)  # Remove spaces
    return text

def normalize_text_but_keep_vietnamese_alphabet(text: str) -> str:
    text = common_normalize(text)
    for base_char, variations in vietnamese_dict.items():
        for char in variations:
            reversed_vietnamese_dict[char] = base_char

    result = []
    for char in text:
        if char in reversed_vietnamese_dict:
            result.append(reversed_vietnamese_dict[char])
        else:
            result.append(char)

    text = "".join(result)
    text = unicodedata.normalize("NFKC", text)

    text = "".join(c for c in text if not unicodedata.combining(c))  # Remove accents
    text = re.sub(r"\s+", "", text)  # Remove spaces
    return text

def normalize_text_but_keep_accent(text: str) -> str:
    """Normalize text by removing accents, spaces, and special cases."""
    text = common_normalize(text)
    text = unicodedata.normalize("NFKC", text)

    for wrong, correct in wrong_accents.items():
        text = text.replace(wrong, correct)

    return text

def segment_text(s, safe=True):
    text = s[:]

    prefixes = SAFE_CASES if safe else ALL_PREFIXES

    for p in prefixes:
        text = re.sub(re.escape(p), ',', text, flags=re.IGNORECASE)

    # Xử lý dấu "-" trong tên (ví dụ: "Ng-T-" -> "Ng T ")
    text = re.sub(r'[.\-]', ' ', text)

    # Tách cụm địa chỉ
    segments = [seg.strip() for seg in text.split(',') if seg.strip()]
    #
    # print()
    # print(f"'{s}'  -->  {segments}")
    return segments


# print(normalize_text_but_keep_accent("T18,Cẩm Bình, Cẩm Phả, Quảng Ninh"))
# print(normalize_text_but_keep_vietnamese("Thôn Đồng Lực Hoàng Lâu, Tam Dương, Vĩnh Phúc"))
# print(normalize_text_but_keep_vietnamese("Tam Đường, Tam Đường, Lai Châu"))



# Index Analyzer

In [10]:

from queue import Queue
from typing import Optional, Tuple, Dict, List

# from Utils import *
locality_map = {}
current_id = 0

# SPECIAL_CASES = ["xã", "x.", "huyện", "tỉnh", "t.",
#                  "tp", "thành phố", "thànhphố"]

# Prefixes for wards and districts to expand possible matches
# DIGIT_CASES = {
#     "ward": ["p", "phường"],
#     "district": ["q", "quận"],
# }
SPECIAL_CASES = ["xã", "x.", "huyện", "tỉnh", "t.", "số nhà", "thị trấn", "ấp",
                 "h.", "khu phố","tp", "thành phố", "thànhphố"]
WARD_CASES = ["p", "p.", "phường"]
DISTRICT_CASES = ["q", "q.", "quận"]

# Dictionary to store generated variations for tracing back
variation_map: Dict[str, dict] = {}

province_short_form = {
    "hồchíminh":"hcm",
    "bàrịavũngtàu":"brvt",
}

class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False
        self.original_string: Optional[str] = None


class Trie:

    def __init__(self):
        self.root = TrieNode()
        self.all_words = set()
        self.original_names: Dict[str, str] = {} #normalize -> raw

    def insert(self, normalized_word: str):
        """Insert a normalized word into the trie with a reference to the original."""
        node = self.root
        for i, char in enumerate(normalized_word):

            if char not in node.children:
                node.children[char] = TrieNode()

            node = node.children[char]

        node.is_end_of_word = True
        node.original_string = normalized_word

        self.all_words.add(normalized_word)

    def search(self, text: str, start: int) -> Optional[Tuple[str, int, int]]:
        """Finds the first valid word from the given start index."""
        node = self.root
        for i in range(start, len(text)):
            char = text[i]
            if char not in node.children:
                break
            node = node.children[char]
            if node.is_end_of_word:
                return (node.original_string, start, i + 1)
        return None

    def search_max_length(self, text: str, start: int) -> Optional[Tuple[str, int, int]]:
        """Finds the longest valid word from the given start index."""
        node = self.root
        longest_match = ""
        longest_length = 0
        for i in range(start, len(text)):
            char = text[i]
            if char not in node.children:
                break
            node = node.children[char]
            if node.is_end_of_word:
                current_length = i - start + 1
                if current_length > longest_length:
                    longest_length = current_length
                    longest_match = (node.original_string, start, i + 1)
        return longest_match

    def get_raw_text(self, normalized_text):
        return self.original_names.get(normalized_text, normalized_text)

def generate_prefixed_variations(location_name: str, category: str) -> Tuple[List[str], str]:
    """Generate prefixed variations ONLY for wards and districts, and store variations per category."""
    variations = []

    if category not in variation_map:
        variation_map[category] = {}

    normalized_name = normalize_text_but_keep_accent(location_name)

    if normalized_name.isdigit():  # Only generate prefixes for wards and districts
        variations = [prefix + normalized_name for prefix in DIGIT_CASES[category]]
    elif normalized_name in province_short_form:
        variations = [normalized_name, province_short_form[normalized_name]]
    else:
        variations = [normalized_name]

    non_accents_variations = [normalize_text_and_remove_accent(variation) for variation in variations]
    variations.extend(non_accents_variations)

    # Store variations per category
    variation_map[category][normalized_name] = variations
    return variations, normalized_name


def load_databases(filenames: Dict[str, str], tries) -> Dict[str, Trie]:
    """Load multiple database files into separate Tries with prefixed variations."""
    for category, filename in filenames.items():
        trie = Trie()
        try:
            with open(filename, "r", encoding="utf-8") as file:
                for line in file:
                    load_line(line, trie, category)
            tries[category] = trie
        except FileNotFoundError:
            print(f"Warning: File {filename} not found!")
            tries[category] = Trie()
    return tries

def load_line(line, trie, category):
    location_name = line.strip()
    if location_name == "":
        return

    prefixed_variations, normalized_text = generate_prefixed_variations(location_name, category)

    for variant in prefixed_variations:
        trie.original_names[variant] = location_name
        trie.insert(variant)


# Autocorrect

In [11]:
# Both Normalized and Un-normalized words are loaded to Trie
# Inputs are misspelled words (misspelled_words)
# Outputs are corrected words via Database
# If Output is 'null', please check Database if it really exists
# Two parameters can be tuned: COSINE_SIMILARITY_THRESHOLD = 0.75
#                                     distance = damerau_levenshtein(word_normalized, candidate_normalized, max_distance=3)
# Search Priority is set as province > district > ward

####################################################################
import time
from collections import Counter
import math

import editdistance

# from IndexAnalyzer import Trie
# from Utils import normalize_text_but_keep_vietnamese_alphabet, normalize_text_but_keep_accent

# Define category ranking
CATEGORY_PRIORITY = {"province": 1, "district": 2, "ward": 3}  # Lower number = higher priority
COSINE_SIMILARITY_THRESHOLD = 0.75  # If similarity < 0.7, return "null", the less value - the less strict
MAX_VALID_EDIT_DISTANCE = 3

# Damerau-Levenshtein Distance with max_distance
# def damerau_levenshtein(s1, s2, max_distance=3):
#     len_s1, len_s2 = len(s1), len(s2)
#     if abs(len_s1 - len_s2) > max_distance:
#         return max_distance + 1
#
#     prev_row = list(range(len_s2 + 1))
#     curr_row = [0] * (len_s2 + 1)
#
#     for i in range(1, len_s1 + 1):
#         curr_row[0] = i
#         min_row_value = i
#
#         for j in range(1, len_s2 + 1):
#             cost = 0 if s1[i - 1] == s2[j - 1] else 1
#             curr_row[j] = min(curr_row[j - 1] + 1, prev_row[j] + 1, prev_row[j - 1] + cost)
#             if i > 1 and j > 1 and s1[i - 1] == s2[j - 2] and s1[i - 2] == s2[j - 1]:
#                 curr_row[j] = min(curr_row[j], prev_row[j - 2] + 1)
#
#             min_row_value = min(min_row_value, curr_row[j])
#
#         if min_row_value > max_distance:
#             return max_distance + 1
#
#         prev_row, curr_row = curr_row, prev_row
#
#     return prev_row[len_s2]

# Cosine Similarity Function
def cosine_similarity(s1, s2):
    vec1, vec2 = Counter(s1), Counter(s2)
    intersection = set(vec1.keys()) & set(vec2.keys())
    dot_product = sum(vec1[x] * vec2[x] for x in intersection)
    magnitude1 = math.sqrt(sum(vec1[x] ** 2 for x in vec1.keys()))
    magnitude2 = math.sqrt(sum(vec2[x] ** 2 for x in vec2.keys()))
    return dot_product / (magnitude1 * magnitude2) if magnitude1 and magnitude2 else 0.0

# Autocorrection with Category Priority, Damerau-Levenshtein Distance & Cosine Similarity Check
def autocorrect(word_normalized, trie: Trie, category):
    best_distance = float("inf")
    matches = []
    for candidate_normalized in trie.all_words:
        distance = editdistance.distance(word_normalized, candidate_normalized)

        # Prioritize lower edit distance & higher-ranked category (Province > District > Ward)
        if distance < min(best_distance, MAX_VALID_EDIT_DISTANCE):
            matches = [candidate_normalized]
            best_distance = distance
        elif distance == min(best_distance, MAX_VALID_EDIT_DISTANCE):
            matches.append(candidate_normalized)

    # Check Cosine Similarity Threshold
    best_match = ""
    best_similarity = 0
    for match in matches:
        p = cosine_similarity(word_normalized, match)
        if p > COSINE_SIMILARITY_THRESHOLD and  p > best_similarity:
            best_similarity = p
            best_match = match

    return best_match


# Searcher

In [12]:
from typing import Dict, Tuple, Optional

# from IndexAnalyzer import Trie
# from Autocorrect import autocorrect
# from Utils import normalize_text_and_remove_accent


def search_locations_in_trie(tries: Dict[str, Trie], input_text: str, results) -> Tuple[Dict[str, Optional[str]], str]:
    matched_positions = set()

    remaining_chars = list(input_text)

    for category, reverse in [("province", True), ("ward", False), ("district", False)]:
        if results[category] != "":
            continue
        match = search_part(tries[category], input_text, matched_positions, remaining_chars, reverse)
        res = match[0]
        if match and res:
            input_text = input_text[:match[1]] + "," + input_text[match[2]:]
        results[category] = res

    # remaining_text = normalize_text_and_remove_accent(input_text)
    # for category, reverse in [("province", True), ("ward", False), ("district", False)]:
    #     if results[category] != "":
    #         continue
    #     match = search_part(tries[category], remaining_text, matched_positions, remaining_chars, reverse)
    #     res = match[0]
    #     if match and res:
    #         remaining_text = remaining_text[:match[1]] + "," + remaining_text[match[2]:]
    #         input_text = input_text[:match[1]] + "," + input_text[match[2]:]
    #     results[category] = res

    return results, input_text

def search_locations_in_segments(tries: Dict[str, Trie], segments: [], results) -> Tuple[Dict[str, Optional[str]], str]:
    for category, reverse in [("province", True), ("ward", False), ("district", False)]:
        if results[category] != "":
            continue

        res = search_in_segment(segments, tries[category], category, reverse)
        results[category] = res

    return results, segments

def search_in_trie(trie, input_text, matched_positions, remaining_chars, reverse):
    match = search_part(trie, input_text, matched_positions, remaining_chars, reverse)
    res = match[0]
    if match and res:
        input_text = input_text[:match[1]] + "," + input_text[match[2]:]
    return res, input_text

def search_in_segment(segments, trie, category, reverse=False):
    if reverse:
        segments.reverse()

    for seg in segments:
        word = autocorrect(seg, trie, category)
        if word == "":
            continue
        segments.remove(seg)
        return word

    if reverse:
        segments.reverse()

    return ""

def search_part(trie, input_text, matched_positions, remaining_chars, reversed=False):
    it_range = range(len(input_text))
    if reversed:
        it_range = it_range.__reversed__()
    for i in it_range:
        if i in matched_positions:
            continue
        match = trie.search_max_length(input_text, i)
        if match:
            return match
    return "", None, None

# Solution

In [13]:
# # NOTE: you MUST change this cell
# # New methods / functions must be written under class Solution.
# class TrieNode:
#     def __init__(self):
#         self.children = {}
#         self.is_end_of_word = False
#         self.original_string: Optional[str] = None

# class Trie:
#     def __init__(self):
#         self.root = TrieNode()
#         self.all_words = set()
#         self.original_names: Dict[str, str] = {}

#     def insert(self, normalized_word: str):
#         """Insert a normalized word into the trie with a reference to the original."""
#         node = self.root
#         for i, char in enumerate(normalized_word):
#             if char not in node.children:
#                 node.children[char] = TrieNode()
#             node = node.children[char]
#         node.is_end_of_word = True
#         node.original_string = normalized_word
#         self.all_words.add(normalized_word)

#     def search(self, text: str, start: int) -> Optional[Tuple[str, int, int]]:
#         """Finds the first valid word from the given start index."""
#         node = self.root
#         for i in range(start, len(text)):
#             char = text[i]
#             if char not in node.children:
#                 break
#             node = node.children[char]
#             if node.is_end_of_word:
#                 return (node.original_string, start, i + 1)
#         return None

#     def search_max_length(self, text: str, start: int) -> Optional[Tuple[str, int, int]]:
#         """Finds the longest valid word from the given start index."""
#         node = self.root
#         longest_match = ""
#         longest_length = 0
#         for i in range(start, len(text)):
#             char = text[i]
#             if char not in node.children:
#                 break
#             node = node.children[char]
#             if node.is_end_of_word:
#                 current_length = i - start + 1
#                 if current_length > longest_length:
#                     longest_length = current_length
#                     longest_match = (node.original_string, start, i + 1)
#         return longest_match

#     def get_raw_text(self, normalized_text):
#         return self.original_names.get(normalized_text, normalized_text)


# # ========================= Solution.py =========================
# class Solution:
#     SAFE_CASES = [
#             "Tỉnh", " T ",
#             "Huyện", "hyện",
#             "Thị xã", "ThịXã", "Xã",
#             "Thị trấn", "ThịTrấn",
#             "khu phố", "KP", "KhuPhố", "Khu pho", "KhuPho",
#         ]

#     ALL_PREFIXES = ["TP.", "T.P", "F.", "Thành phố", "ThànhPhố", "TP ", " TP",
#                                 "Tỉnh", "Tỉn", "T.", " T ",
#                                 "Quận", "Q.", " Q ", ",Q ",
#                                 "Huyện", "hyện", "H.", " H ",
#                                 "khu phố", "KhuPhố", "Khu pho", "KhuPho",
#                                 "Phường", "P.", "F", " P ",
#                                 "X.", "Thị xã", "ThịXã", "Xã",
#                                 "Thị trấn", "ThịTrấn", "T.T",
#                                 "-"
#                                 ]
#     vietnamese_dict = {
#                 "a": ["à", "á", "ạ", "ả", "ã", "â", "ầ", "ấ", "ậ", "ẩ", "ẫ", "ă", "ằ", "ắ", "ặ", "ẳ", "ẵ"],
#                 "d": ["đ"],
#                 "e": ["è", "é", "ẹ", "ẻ", "ẽ", "ê", "ề", "ế", "ệ", "ể", "ễ"],
#                 "i": ["ì", "í", "ị", "ỉ", "ĩ"],
#                 "o": ["ò", "ó", "ọ", "ỏ", "õ", "ô", "ồ", "ố", "ộ", "ổ", "ỗ", "ơ", "ờ", "ớ", "ợ", "ở", "ỡ"],
#                 "u": ["ù", "ú", "ụ", "ủ", "ũ", "ư", "ừ", "ứ", "ự", "ử", "ữ"],
#                 "y": ["ỳ", "ý", "ỵ", "ỷ", "ỹ"],
#             }

#     wrong_accents = {
#                 "oà": "òa", "oá": "óa", "oạ": "ọa", "oã": "õa", "oả": "ỏa",
#                 "qui": "quy",
#             }

#     SPECIAL_CASES = ["xã", "x.", "huyện", "tỉnh", "t.",
#                     "tp", "thành phố", "thànhphố"]

#     # Prefixes for wards and districts to expand possible matches
#     DIGIT_CASES = {
#         "ward": ["p", "phường"],
#         "district": ["q", "quận"],
#     }

#     # Dictionary to store generated variations for tracing back
#     variation_map: Dict[str, dict] = {}

#     province_short_form = {
#         "hồchíminh":"hcm",
#         "bàrịavũngtàu":"brvt",
#     }

#     CATEGORY_PRIORITY = {"province": 1, "district": 2, "ward": 3}  # Lower number = higher priority
#     COSINE_SIMILARITY_THRESHOLD = 0.75  # If similarity < 0.7, return "null", the less value - the less strict
#     MAX_VALID_EDIT_DISTANCE = 3
#     def __init__(self):
#         # List for province, district, ward for private test, do not change for any reason (these files will be provided later with this exact name)
#         self.province_path = 'list_province.txt'
#         self.district_path = 'list_district.txt'
#         self.ward_path = 'list_ward.txt'

#         self.tries = {}
#         self.variation_map = {}
#         self.locality_map = {}
#         self.current_id = 0
#         self.reversed_vietnamese_dict = {}


#         self.load_databases({
#             "province": self.province_path,
#             "district": self.district_path,
#             "ward": self.ward_path
#         })

#     def common_normalize(self, text: str) -> str:
#         text = text.lower()
#         for case in self.SAFE_CASES:
#             text = re.sub(re.escape(case), ',', text, flags=re.IGNORECASE)
#         text = re.sub(r"\s{2,}", ",", text)  # Remove spaces
#         text = text.replace(" ", "")
#         return text

#     def normalize_text_and_remove_accent(self, text: str) -> str:
#         """Normalize text by removing accents, spaces, and special cases."""
#         text = self.common_normalize(text)
#         text = unicodedata.normalize("NFKD", text)
#         text = text.replace("đ", "d")
#         text = "".join(c for c in text if not unicodedata.combining(c))  # Remove accents
#         return text

#     def normalize_text_but_keep_vietnamese_alphabet(self, text: str) -> str:
#         text = self.common_normalize(text)
#         for base_char, variations in vietnamese_dict.items():
#             for char in variations:
#                 self.reversed_vietnamese_dict[char] = base_char

#         result = []
#         for char in text:
#             if char in self.reversed_vietnamese_dict:
#                 result.append(self.reversed_vietnamese_dict[char])
#             else:
#                 result.append(char)

#         text = "".join(result)
#         text = unicodedata.normalize("NFKC", text)

#         text = "".join(c for c in text if not unicodedata.combining(c))  # Remove accents
#         text = re.sub(r"\s+", "", text)  # Remove spaces
#         return text

#     def normalize_text_but_keep_accent(self, text: str) -> str:
#         """Normalize text by removing accents, spaces, and special cases."""
#         text = self.common_normalize(text)
#         text = unicodedata.normalize("NFKC", text)

#         for wrong, correct in wrong_accents.items():
#             text = text.replace(wrong, correct)

#         return text

#     def segment_text(self, s, safe=True):
#         text = s[:]

#         prefixes = self.SAFE_CASES if safe else self.ALL_PREFIXES

#         for p in prefixes:
#             text = re.sub(re.escape(p), ',', text, flags=re.IGNORECASE)

#         # Handle "-" in names (e.g., "Ng-T-" -> "Ng T ")
#         text = re.sub(r'[.\-]', ' ', text)

#         # Split address components
#         segments = [seg.strip() for seg in text.split(',') if seg.strip()]
#         return segments

#     def generate_prefixed_variations(self, location_name: str, category: str) -> Tuple[List[str], str]:
#         """Generate prefixed variations ONLY for wards and districts, and store variations per category."""
#         variations = []

#         if category not in self.variation_map:
#           self.variation_map[category] = {}

#         normalized_name = self.normalize_text_but_keep_accent(location_name)

#         if normalized_name.isdigit():  # Only generate prefixes for wards and districts
#             variations = [prefix + normalized_name for prefix in self.DIGIT_CASES[category]]
#         elif normalized_name in province_short_form:
#             variations = [normalized_name, province_short_form[normalized_name]]
#         else:
#             variations = [normalized_name]

#         non_accents_variations = [self.normalize_text_and_remove_accent(variation) for variation in variations]
#         variations.extend(non_accents_variations)

#         # Store variations per category
#         self.variation_map[category][normalized_name] = variations
#         return variations, normalized_name

#     def load_databases(self, filenames: Dict[str, str]) -> None:
#         """Load multiple database files into separate Tries with prefixed variations."""
#         for category, filename in filenames.items():
#             trie = Trie()
#             try:
#                 with open(filename, "r", encoding="utf-8") as file:
#                     for line in file:
#                         self.load_line(line, trie, category)
#                 self.tries[category] = trie
#             except FileNotFoundError:
#                 print(f"Warning: File {filename} not found!")
#                 self.tries[category] = Trie()

#     def load_line(self, line, trie, category):
#         location_name = line.strip()
#         if location_name == "":
#             return

#         prefixed_variations, normalized_text = self.generate_prefixed_variations(location_name, category)

#         for variant in prefixed_variations:
#             trie.original_names[variant] = location_name
#             trie.insert(variant)

#     # Cosine Similarity Function
#     def cosine_similarity(self,s1, s2):
#         vec1, vec2 = Counter(s1), Counter(s2)
#         intersection = set(vec1.keys()) & set(vec2.keys())
#         dot_product = sum(vec1[x] * vec2[x] for x in intersection)
#         magnitude1 = math.sqrt(sum(vec1[x] ** 2 for x in vec1.keys()))
#         magnitude2 = math.sqrt(sum(vec2[x] ** 2 for x in vec2.keys()))
#         return dot_product / (magnitude1 * magnitude2) if magnitude1 and magnitude2 else 0.0

#         # Autocorrection with Category Priority, Damerau-Levenshtein Distance & Cosine Similarity Check
#     def autocorrect(self, word_normalized, trie, category):
#         best_distance = float("inf")
#         matches = []
#         for candidate_normalized in trie.all_words:
#             distance = editdistance.distance(word_normalized, candidate_normalized)

#             # Prioritize lower edit distance & higher-ranked category (Province > District > Ward)
#             if distance < min(best_distance, MAX_VALID_EDIT_DISTANCE):
#                 matches = [candidate_normalized]
#                 best_distance = distance
#             elif distance == min(best_distance, MAX_VALID_EDIT_DISTANCE):
#                 matches.append(candidate_normalized)

#         # Check Cosine Similarity Threshold
#         best_match = ""
#         best_similarity = 0
#         for match in matches:
#             p = self.cosine_similarity(word_normalized, match)
#             if p > COSINE_SIMILARITY_THRESHOLD and p > best_similarity:
#                 best_similarity = p
#                 best_match = match

#         return best_match

#     def search_locations_in_trie(self, tries, input_text, results):
#         matched_positions = set()
#         remaining_chars = list(input_text)

#         for category, reverse in [("province", True), ("ward", False), ("district", False)]:
#             if results[category] != "":
#                 continue
#             match = self.search_part(tries[category], input_text, matched_positions, remaining_chars, reverse)
#             res = match[0]
#             if match and res:
#                 input_text = input_text[:match[1]] + "," + input_text[match[2]:]
#             results[category] = res

#         return results, input_text

#     def search_locations_in_segments(self, tries, segments, results):
#         for category, reverse in [("province", True), ("ward", False), ("district", False)]:
#             if results[category] != "":
#                 continue

#             res = self.search_in_segment(segments, tries[category], category, reverse)
#             results[category] = res

#         return results, segments

#     def search_in_trie(self, trie, input_text, matched_positions, remaining_chars, reverse):
#         match = self.search_part(trie, input_text, matched_positions, remaining_chars, reverse)
#         res = match[0]
#         if match and res:
#             input_text = input_text[:match[1]] + "," + input_text[match[2]:]
#         return res, input_text

#     def search_in_segment(self, segments, trie, category, reverse=False):
#         if reverse:
#             segments.reverse()

#         for seg in segments:
#             word = self.autocorrect(seg, trie, category)
#             if word == "":
#                 continue
#             segments.remove(seg)
#             return word

#         if reverse:
#             segments.reverse()

#         return ""

#     def search_part(self, trie, input_text, matched_positions, remaining_chars, reversed=False):
#         it_range = range(len(input_text))
#         if reversed:
#             it_range = it_range.__reversed__()
#         for i in it_range:
#             if i in matched_positions:
#                 continue
#             match = trie.search_max_length(input_text, i)
#             if match:
#                 return match
#         return "", None, None

#     def process(self, s: str):
#       # Ghi lại chuỗi đầu vào ban đầu
#       s_copy = s[:]

#       # Tiến hành chia tách và chuẩn hóa chuỗi
#       segments = self.segment_text(s)  # Tách các phần của địa chỉ
#       input_text = self.normalize_text_but_keep_accent(",".join(segments))  # Chuẩn hóa mà không loại bỏ dấu tiếng Việt

#       # Khởi tạo kết quả ban đầu cho ward, district, province
#       results = {"ward": "", "district": "", "province": ""}

#       # Tìm kiếm trong trie với dấu
#       result, remaining_text = self.search_locations_in_trie(self.tries, input_text, results)

#       # Nếu không tìm thấy kết quả ở tỉnh/quận/phường, thử tìm kiếm lại không có dấu
#       if result["province"] == "" and result["district"] == "" and result["ward"] == "":
#           remaining_text = self.normalize_text_and_remove_accent(remaining_text)
#           result, remaining_text = self.search_locations_in_trie(self.tries, remaining_text, results)

#       # Nếu vẫn chưa tìm thấy, thử tìm kiếm theo các phần của chuỗi (segments)
#       if result["province"] == "" and result["district"] == "" and result["ward"] == "":
#           segments = self.segment_text(remaining_text, False)
#           result, remaining_text = self.search_locations_in_segments(self.tries, segments, results)


#       return {
#           "province": self.tries["province"].get_raw_text(result["province"]),
#           "district": self.tries["district"].get_raw_text(result["district"]),
#           "ward": self.tries["ward"].get_raw_text(result["ward"]),
#       }

#     def debug_address(self, address: str):
#         """Debugging function to print step-by-step."""
#         print("\n📌 Input Address:", address)

#         # Step 1: Normalize the address (remove spaces, special cases, etc.)
#         preprocessed = self.normalize_text_but_keep_accent(address)
#         print("🔹 After Preprocessing (with accents):", preprocessed)

#         # Step 2: Segment the address into components (e.g., province, district, ward)
#         segments = self.segment_text(address)
#         print("🔹 Address Segments:", segments)

#         # Step 3: Search for matches in the trie (with accents)
#         results = {"ward": "", "district": "", "province": ""}
#         matched_results, remaining_text = self.search_locations_in_trie(self.tries, preprocessed, results)
#         print("🔹 Extracted Matches (with accents):", matched_results)
#         print("🔹 Remaining Text After Accent Search:", remaining_text)

#         # Step 4: If no match found, try to remove accents and search again
#         if matched_results["province"] == "" and matched_results["district"] == "" and matched_results["ward"] == "":
#             remaining_text_no_accents = self.normalize_text_and_remove_accent(remaining_text)
#             matched_results_no_accents, remaining_text = self.search_locations_in_trie(self.tries, remaining_text_no_accents, results)
#             print("🔹 Extracted Matches (without accents):", matched_results_no_accents)
#             print("🔹 Remaining Text After No-Accents Search:", remaining_text)

#         # Step 5: If still no matches, try searching by individual segments
#         if matched_results["province"] == "" and matched_results["district"] == "" and matched_results["ward"] == "":
#             segments = self.segment_text(remaining_text, False)
#             matched_results, remaining_text = self.search_locations_in_segments(self.tries, segments, results)
#             print("🔹 Matches After Searching by Segments:", matched_results)
#             print("🔹 Remaining Text After Searching by Segments:", remaining_text)

#         # Step 6: Unnormalize the final results (convert normalized names back to original form)
#         final_results = {
#             "province": self.tries["province"].get_raw_text(matched_results["province"]),
#             "district": self.tries["district"].get_raw_text(matched_results["district"]),
#             "ward": self.tries["ward"].get_raw_text(matched_results["ward"]),
#         }

#         print("✅ Final Address Components:", final_results)
#         return final_results


class Solution:

    debug=False

    def __init__(self):
        # list provice, district, ward for private test, do not change for any reason (these file will be provided later with this exact name)

        self.province_path = 'list_province.txt'
        self.district_path = 'list_district.txt'
        self.ward_path = 'list_ward.txt'

        self.tries = {}
        load_databases({
            "province": self.province_path,
            "district": self.district_path,
            "ward": self.ward_path
        }, self.tries)

        self.variation_map = variation_map
        pass

    def process(self, s: str):
        # Preprocess
        s_copy = s[:]

        segments = segment_text(s)
        input_text = normalize_text_but_keep_accent(",".join(segments))

        # Start searching
        results = {"ward": "", "district": "", "province": ""}

        # Search with accents
        result, remaining_text = search_locations_in_trie(self.tries, input_text, results)

        # If the province/district/ward not found, search without accents
        # remaining_text = normalize_text_and_remove_accent(remaining_text)
        # result, remaining_text = search_locations_in_trie(self.tries, remaining_text, results)

        # If the province/district/ward not found, search by segments
        segments = segment_text(remaining_text, False)
        result, remaining_text = search_locations_in_segments(self.tries, segments, results)

        result =  {
            "province": self.tries["province"].get_raw_text(result["province"]),
            "district": self.tries["district"].get_raw_text(result["district"]),
            "ward": self.tries["ward"].get_raw_text(result["ward"]),
        }

        if self.debug:
            print()
            print(f"Original: {s_copy}")
            print(f"Normalized: {normalize_text_but_keep_accent(s_copy)}")
            print(f"Result: {result}")

        return result

# Điền Hải -> Điên Hải -> Tiên Hải
# runner = Solution()
# runner.debug = True

# runner.process("Diên Thạnh,,T Khabnh Hòa")


# Not able to solve yet
# runner.process(" T.P Phan Rang-Tháp lhàm  Ninh Thuận")
# runner.process("Điên Hải, Đông Hải, T bạc Liêu")



# Run tests

In [14]:
# CORRECT TESTS
groups_province = {}
groups_district = {'hòa bình': ['Hoà Bình', 'Hòa Bình'], 'kbang': ['Kbang', 'KBang'], 'quy nhơn': ['Qui Nhơn', 'Quy Nhơn']}
groups_ward = {'ái nghĩa': ['ái Nghĩa', 'Ái Nghĩa'], 'ái quốc': ['ái Quốc', 'Ái Quốc'], 'ái thượng': ['ái Thượng', 'Ái Thượng'], 'ái tử': ['ái Tử', 'Ái Tử'], 'ấm hạ': ['ấm Hạ', 'Ấm Hạ'], 'an ấp': ['An ấp', 'An Ấp'], 'ẳng cang': ['ẳng Cang', 'Ẳng Cang'], 'ẳng nưa': ['ẳng Nưa', 'Ẳng Nưa'], 'ẳng tở': ['ẳng Tở', 'Ẳng Tở'], 'an hòa': ['An Hoà', 'An Hòa'], 'ayun': ['Ayun', 'AYun'], 'bắc ái': ['Bắc ái', 'Bắc Ái'], 'bảo ái': ['Bảo ái', 'Bảo Ái'], 'bình hòa': ['Bình Hoà', 'Bình Hòa'], 'châu ổ': ['Châu ổ', 'Châu Ổ'], 'chư á': ['Chư á', 'Chư Á'], 'chư rcăm': ['Chư Rcăm', 'Chư RCăm'], 'cộng hòa': ['Cộng Hoà', 'Cộng Hòa'], 'cò nòi': ['Cò  Nòi', 'Cò Nòi'], 'đại ân 2': ['Đại Ân  2', 'Đại Ân 2'], 'đak ơ': ['Đak ơ', 'Đak Ơ'], "đạ m'ri": ["Đạ M'ri", "Đạ M'Ri"], 'đông hòa': ['Đông Hoà', 'Đông Hòa'], 'đồng ích': ['Đồng ích', 'Đồng Ích'], 'hải châu i': ['Hải Châu  I', 'Hải Châu I'], 'hải hòa': ['Hải Hoà', 'Hải Hòa'], 'hành tín đông': ['Hành Tín  Đông', 'Hành Tín Đông'], 'hiệp hòa': ['Hiệp Hoà', 'Hiệp Hòa'], 'hòa bắc': ['Hoà Bắc', 'Hòa Bắc'], 'hòa bình': ['Hoà Bình', 'Hòa Bình'], 'hòa châu': ['Hoà Châu', 'Hòa Châu'], 'hòa hải': ['Hoà Hải', 'Hòa Hải'], 'hòa hiệp trung': ['Hoà Hiệp Trung', 'Hòa Hiệp Trung'], 'hòa liên': ['Hoà Liên', 'Hòa Liên'], 'hòa lộc': ['Hoà Lộc', 'Hòa Lộc'], 'hòa lợi': ['Hoà Lợi', 'Hòa Lợi'], 'hòa long': ['Hoà Long', 'Hòa Long'], 'hòa mạc': ['Hoà Mạc', 'Hòa Mạc'], 'hòa minh': ['Hoà Minh', 'Hòa Minh'], 'hòa mỹ': ['Hoà Mỹ', 'Hòa Mỹ'], 'hòa phát': ['Hoà Phát', 'Hòa Phát'], 'hòa phong': ['Hoà Phong', 'Hòa Phong'], 'hòa phú': ['Hoà Phú', 'Hòa Phú'], 'hòa phước': ['Hoà Phước', 'Hòa Phước'], 'hòa sơn': ['Hoà Sơn', 'Hòa Sơn'], 'hòa tân': ['Hoà Tân', 'Hòa Tân'], 'hòa thuận': ['Hoà Thuận', 'Hòa Thuận'], 'hòa tiến': ['Hoà Tiến', 'Hòa Tiến'], 'hòa trạch': ['Hoà Trạch', 'Hòa Trạch'], 'hòa vinh': ['Hoà Vinh', 'Hòa Vinh'], 'hương hòa': ['Hương Hoà', 'Hương Hòa'], 'ích hậu': ['ích Hậu', 'Ích Hậu'], 'ít ong': ['ít Ong', 'Ít Ong'], 'khánh hòa': ['Khánh Hoà', 'Khánh Hòa'], 'krông á': ['Krông Á', 'KRông á'], 'lộc hòa': ['Lộc Hoà', 'Lộc Hòa'], 'minh hòa': ['Minh Hoà', 'Minh Hòa'], 'mường ải': ['Mường ải', 'Mường Ải'], 'mường ẳng': ['Mường ẳng', 'Mường Ẳng'], 'nậm ét': ['Nậm ét', 'Nậm Ét'], 'nam hòa': ['Nam Hoà', 'Nam Hòa'], 'na ư': ['Na ư', 'Na Ư'], 'ngã sáu': ['Ngã sáu', 'Ngã Sáu'], 'nghi hòa': ['Nghi Hoà', 'Nghi Hòa'], 'nguyễn úy': ['Nguyễn Uý', 'Nguyễn úy', 'Nguyễn Úy'], 'nhân hòa': ['Nhân Hoà', 'Nhân Hòa'], 'nhơn hòa': ['Nhơn Hoà', 'Nhơn Hòa'], 'nhơn nghĩa a': ['Nhơn nghĩa A', 'Nhơn Nghĩa A'], 'phúc ứng': ['Phúc ứng', 'Phúc Ứng'], 'phước hòa': ['Phước Hoà', 'Phước Hòa'], 'sơn hóa': ['Sơn Hoá', 'Sơn Hóa'], 'tạ an khương đông': ['Tạ An Khương  Đông', 'Tạ An Khương Đông'], 'tạ an khương nam': ['Tạ An Khương  Nam', 'Tạ An Khương Nam'], 'tăng hòa': ['Tăng Hoà', 'Tăng Hòa'], 'tân hòa': ['Tân Hoà', 'Tân Hòa'], 'tân hòa thành': ['Tân Hòa  Thành', 'Tân Hòa Thành'], 'tân khánh trung': ['Tân  Khánh Trung', 'Tân Khánh Trung'], 'tân lợi': ['Tân lợi', 'Tân Lợi'], 'thái hòa': ['Thái Hoà', 'Thái Hòa'], 'thiết ống': ['Thiết ống', 'Thiết Ống'], 'thuận hòa': ['Thuận Hoà', 'Thuận Hòa'], 'thượng ấm': ['Thượng ấm', 'Thượng Ấm'], 'thụy hương': ['Thuỵ Hương', 'Thụy Hương'], 'thủy xuân': ['Thuỷ Xuân', 'Thủy Xuân'], 'tịnh ấn đông': ['Tịnh ấn Đông', 'Tịnh Ấn Đông'], 'tịnh ấn tây': ['Tịnh ấn Tây', 'Tịnh Ấn Tây'], 'triệu ái': ['Triệu ái', 'Triệu Ái'], 'triệu ẩu': ['Triệu ẩu', 'Triệu Ẩu'], 'trung hòa': ['Trung Hoà', 'Trung Hòa'], 'trung ý': ['Trung ý', 'Trung Ý'], 'tùng ảnh': ['Tùng ảnh', 'Tùng Ảnh'], 'úc kỳ': ['úc Kỳ', 'Úc Kỳ'], 'ứng hòe': ['ứng Hoè', 'Ứng Hoè'], 'vĩnh hòa': ['Vĩnh Hoà', 'Vĩnh Hòa'], 'vũ hòa': ['Vũ Hoà', 'Vũ Hòa'], 'xuân ái': ['Xuân ái', 'Xuân Ái'], 'xuân áng': ['Xuân áng', 'Xuân Áng'], 'xuân hòa': ['Xuân Hoà', 'Xuân Hòa'], 'xuất hóa': ['Xuất Hoá', 'Xuất Hóa'], 'ỷ la': ['ỷ La', 'Ỷ La']}
groups_ward.update({1: ['1', '01'], 2: ['2', '02'], 3: ['3', '03'], 4: ['4', '04'], 5: ['5', '05'], 6: ['6', '06'], 7: ['7', '07'], 8: ['8', '08'], 9: ['9', '09']})
def to_same(groups):
    same = {ele: k for k, v in groups.items() for ele in v}
    return same
same_province = to_same(groups_province)
same_district = to_same(groups_district)
same_ward = to_same(groups_ward)
def normalize(text, same_dict):
    return same_dict.get(text, text)

In [15]:
# NOTE: DO NOT change this cell
# This cell is for downloading private test
!rm -rf test.json
# this link is public test
!gdown --fuzzy https://drive.google.com/file/d/1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB/view?usp=sharing -O test.json

Downloading...
From: https://drive.google.com/uc?id=1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB
To: /content/test.json
100% 79.4k/79.4k [00:00<00:00, 61.8MB/s]


In [16]:
debug = True

TEAM_NAME = 'GROUP4'  # This should be your team name
EXCEL_FILE = f'{TEAM_NAME}.xlsx'

import json
import time
with open('test.json') as f:
    data = json.load(f)

summary_only = True
df = []
solution = Solution()
timer = []
correct = 0
for test_idx, data_point in enumerate(data):
    address = data_point["text"]

    ok = 0
    try:
        answer = data_point["result"]
        answer["province_normalized"] = normalize(answer["province"], same_province)
        answer["district_normalized"] = normalize(answer["district"], same_district)
        answer["ward_normalized"] = normalize(answer["ward"], same_ward)

        start = time.perf_counter_ns()
        result = solution.process(address)
        finish = time.perf_counter_ns()
        timer.append(finish - start)
        result["province_normalized"] = normalize(result["province"], same_province)
        result["district_normalized"] = normalize(result["district"], same_district)
        result["ward_normalized"] = normalize(result["ward"], same_ward)

        province_correct = int(answer["province_normalized"] == result["province_normalized"])
        district_correct = int(answer["district_normalized"] == result["district_normalized"])
        ward_correct = int(answer["ward_normalized"] == result["ward_normalized"])
        ok = province_correct + district_correct + ward_correct

        df.append([
            test_idx,
            address,
            answer["province"],
            result["province"],
            answer["province_normalized"],
            result["province_normalized"],
            province_correct,
            answer["district"],
            result["district"],
            answer["district_normalized"],
            result["district_normalized"],
            district_correct,
            answer["ward"],
            result["ward"],
            answer["ward_normalized"],
            result["ward_normalized"],
            ward_correct,
            ok,
            timer[-1] / 1_000_000_000,
        ])
        if debug and ok < 3:
          print()
          print("Original: " + address)
          if province_correct == 0:
              print(f"Province -> Result: '{result['province']}', Answer: '{answer['province']}'")
          if district_correct == 0:
              print(f"District -> Result: '{result['district']}', Answer: '{answer['district']}'")
          if ward_correct == 0:
              print(f"Ward -> Result: '{result['ward']}', Answer: '{answer['ward']}'")

    except Exception as e:
        print(f"{answer = }")
        print(f"{result = }")
        df.append([
            test_idx,
            address,
            answer["province"],
            "EXCEPTION",
            answer["province_normalized"],
            "EXCEPTION",
            0,
            answer["district"],
            "EXCEPTION",
            answer["district_normalized"],
            "EXCEPTION",
            0,
            answer["ward"],
            "EXCEPTION",
            answer["ward_normalized"],
            "EXCEPTION",
            0,
            0,
            0,
        ])
        # any failure count as a zero correct
        pass
    correct += ok


    if not summary_only:
        # responsive stuff
        print(f"Test {test_idx:5d}/{len(data):5d}")
        print(f"Correct: {ok}/3")
        print(f"Time Executed: {timer[-1] / 1_000_000_000:.4f}")


print(f"-"*30)
total = len(data) * 3
score_scale_10 = round(correct / total * 10, 2)
if len(timer) == 0:
    timer = [0]
max_time_sec = round(max(timer) / 1_000_000_000, 4)
avg_time_sec = round((sum(timer) / len(timer)) / 1_000_000_000, 4)

import pandas as pd

df2 = pd.DataFrame(
    [[correct, total, score_scale_10, max_time_sec, avg_time_sec]],
    columns=['correct', 'total', 'score / 10', 'max_time_sec', 'avg_time_sec',],
)

columns = [
    'ID',
    'text',
    'province',
    'province_student',
    'province_normalized',
    'province_student_normalized',
    'province_correct',
    'district',
    'district_student',
    'district_normalized',
    'district_student_normalized',
    'district_correct',
    'ward',
    'ward_student',
    'ward_normalized',
    'ward_student_normalized',
    'ward_correct',
    'total_correct',
    'time_sec',
]

df = pd.DataFrame(df)
df.columns = columns

print(df2)

!pip install xlsxwriter

if not debug:
    print(f'{TEAM_NAME = }')
    print(f'{EXCEL_FILE = }')
    writer = pd.ExcelWriter(EXCEL_FILE, engine='xlsxwriter')
    df2.to_excel(writer, index=False, sheet_name='summary')
    df.to_excel(writer, index=False, sheet_name='details')
    writer.close()


# Thái Hòa, Hợp Đồng Chương Mỹ, Hà Nội





Original: 357/28,Ng-T- Thuật,P1,Q3,TP.HồChíMinh.
Ward -> Result: '12', Answer: ''

Original: 284DBis Ng Văn Giáo, P3, Mỹ Tho, T.Giang.
Province -> Result: 'An Giang', Answer: 'Tiền Giang'

Original: Tiểu khu 3, thị trấn Ba Hàng, huyện Phổ Yên, tỉnh Thái Nguyên.
Province -> Result: 'Thái Nguyên', Answer: ''
District -> Result: 'Phổ Yên', Answer: ''

Original: 154/4/81 Nguyễn - Phúc Chu, P15, TB, TP. Hồ Chí Minh
Province -> Result: 'Hồ Chí Minh', Answer: ''
Ward -> Result: 'Chợ Chu', Answer: ''

Original: Tổ 8, Ấp An Bình Minh Hòa, Châu Thành, Kiên Giang
Ward -> Result: 'Bình Minh', Answer: 'Minh Hòa'

Original: Phố Đức Sơn, TT Bút Sơn, Hoằng Hoá, Thanh Hoá.
Ward -> Result: 'Đức Sơn', Answer: 'Bút Sơn'

Original: 285 B/1A Bình Gĩa Phường 8,Vũng Tàu,Bà Rịa - Vũng Tàu
Province -> Result: '', Answer: 'Bà Rịa - Vũng Tàu'

Original: KP Phước Trung Thị trấn Đất Đỏ, Đất Đỏ, Bà Rịa - Vũng Tàu
Province -> Result: '', Answer: 'Bà Rịa - Vũng Tàu'

Original: 55/30 Nguyễn - Thượng Hiền,P5,BT, TP. Hồ